In [1]:
import numpy as np
import pandas as pd
import sys,time,math
from numba import jit
import matplotlib.pyplot as plt

In [5]:
# FUNCTIONS
#===========================================================================
# Calculate Crastal Cell Coordinates: (x_i,y_i.z_i), in mm
#----------------------------------------------
@jit
def Cal_Crastal_Sites(PositionNumsXYZ):
    CellNums = PositionNumsXYZ[0]*PositionNumsXYZ[1]*PositionNumsXYZ[2] #(x,y,z)
    Crastal_SizeXYZ = np.array([20,20,20]) # mm  #(x,y,z)

    index = np.arange(CellNums) #Numpy: shape[3] -> shape[2] ->shape[1] -> shape[0]
    index.shape = (PositionNumsXYZ[2],PositionNumsXYZ[1],PositionNumsXYZ[0]) # (NumZ,NumY,NumX) eg (2,6,6)
    X = np.zeros(shape = (CellNums),dtype = float)
    Y = np.zeros(shape = (CellNums),dtype = float)
    Z = np.zeros(shape = (CellNums),dtype = float)
    for i in np.arange(CellNums):
        index_i = np.array(np.where(index == i))
        index_i.shape = (3,)
        #print(index_i)
        X[i] = index_i[2] * (Crastal_SizeXYZ[0] / PositionNumsXYZ[0]) - Crastal_SizeXYZ[0] / 2 + Crastal_SizeXYZ[0] / (2*PositionNumsXYZ[0])
        Y[i] = index_i[1] * (Crastal_SizeXYZ[1] / PositionNumsXYZ[1]) - Crastal_SizeXYZ[1] / 2 + Crastal_SizeXYZ[1] / (2*PositionNumsXYZ[1])
        Z[i] = index_i[0] * (Crastal_SizeXYZ[2] / PositionNumsXYZ[2]) - Crastal_SizeXYZ[2] / 2 + Crastal_SizeXYZ[2] / (2*PositionNumsXYZ[2])
    return X,Y,Z

# Calculate SIPM Coordinates: (x_s,y_s.z_s), in mm
#----------------------------------------------
@jit
def Cal_SIPM_Sites():
    SIPM_gap = 0.29
    SIPM_size = 3.07
    Crastal_size = 20
    Coupling_layer = 0.1
    SIPM_thick = 2
    SIPM_Sites_array_XY = [-2.5*(SIPM_size+SIPM_gap),-1.5*(SIPM_size+SIPM_gap),-0.5*(SIPM_size+SIPM_gap),0.5*(SIPM_size+SIPM_gap),1.5*(SIPM_size+SIPM_gap),2.5*(SIPM_size+SIPM_gap)]
    SIPM_Sites_array_Z = [-(0.5*(Crastal_size + SIPM_thick) + Coupling_layer),0.5*(Crastal_size + SIPM_thick) + Coupling_layer]
    index = np.arange(6*6*2)
    index.shape = (2,6,6) #  (Z,Y,X) X -> Y -> Z 
    X = np.zeros(shape = (6*6*2),dtype = float)
    Y = np.zeros(shape = (6*6*2),dtype = float)
    Z = np.zeros(shape = (6*6*2),dtype = float)
    for s in np.arange(6*6*2):
        index_s = np.array(np.where(index == s))
        index_s.shape = (3,)
        #print(index_s)
        X[s] = SIPM_Sites_array_XY[index_s[2]]
        Y[s] = SIPM_Sites_array_XY[index_s[1]]
        Z[s] = SIPM_Sites_array_Z[index_s[0]]
    return X,Y,Z

#Initializing parameter: mu_is 
#----------------------------------------------
@jit
def Pars_Initialization(Coor_Cells,Coor_SIPMs):
    mu_is = np.zeros(shape = (Coor_Cells.shape[1],Coor_SIPMs.shape[1]),dtype = float)
    for i in range(Coor_Cells.shape[1]):
        for s in range(Coor_SIPMs.shape[1]):
            #print(i,s,Coor_Cells[i][0] , Coor_SIPMs[s][0],Coor_Cells[i][1], Coor_SIPMs[s][1],Coor_Cells[i][2] , Coor_SIPMs[s][2])
            mu_is[i][s] = 1. / ((Coor_Cells[0][i] - Coor_SIPMs[0][s])**2 + (Coor_Cells[1][i] - Coor_SIPMs[1][s])**2 + (Coor_Cells[2][i] - Coor_SIPMs[2][s])**2)
    #print(mu_is)
    Sum_SIPM_i = np.sum(mu_is,axis = 1)
    for i in range(Coor_Cells.shape[1]):
        mu_is[i,:] = mu_is[i,:] / Sum_SIPM_i[i]
    return mu_is

# # The Predicting Function: Inverted_Distri_P
# #----------------------------------------------
# def model_inner(n,Pars_mu,Pars_mu_minus,):
#     for i in np.arange(Pars_mu.shape[0]):
#         GridID_tem[n] = np.sum(Pars_mu_minus**2,axis = 1).argmin()
    

@jit
def model(DataSetA,Pars_mu):
    Process_Num = DataSetA.shape[0]
    GridID_tem = np.zeros(shape = (Process_Num),dtype = float)     # GridID
    Pars_mu_minus = np.zeros(shape = (Pars_mu.shape),dtype = float)
    Inverted_Distri_P = np.zeros(shape = Pars_mu.shape[0],dtype = float)   #IPD
    # Griding GridID
    for n in np.arange(Process_Num):
#         for i in np.arange(Pars_mu.shape[0]):
#             Pars_mu_minus[i,:] = DataSetA[n,:] - Pars_mu[i,:]
        Pars_mu_minus = DataSetA[n,:] - Pars_mu
        GridID_tem[n] = np.sum(Pars_mu_minus**2,axis = 1).argmin() # GridID for Gamma n
        #Inverted_Distri_P[GridID_tem[n].astype(int)] += 1 / Process_Num    #Slow
    # Counting IPD
    Grided_Data = pd.Series(data = GridID_tem)
    GridID_index = np.array(Grided_Data.value_counts().index).astype(int) # IDs for grids has Gammas
    GridID_Distri_P = np.array(Grided_Data.value_counts()) / Process_Num # Normalizing
    
    Inverted_Distri_P[GridID_index] = GridID_Distri_P # p_i

    return GridID_tem,Inverted_Distri_P
             
#
# The Gradient 
# The Object Function 
# f = f1 + f2 + f3
#----------------------------------------------
@jit
def cost(DataSet_A,Real_Distri_P,Pars_mu):
    ProcessNum = DataSet_A.shape[0]
    # Inverting
    Inverted_GridID_tem,Inverted_Distri_P_tem = model(DataSet_A,Pars_mu)   #Inverting
    # Indexing Pars_mu
    Pars_mu_ns = Pars_mu[Inverted_GridID_tem.astype(int),:]     # n*s
    # f1
    Pars_mu_minus1 = DataSet_A -  Pars_mu_ns
    f1 = ((Pars_mu_minus1**2).sum(axis = 1).sum(axis = 0))
    # f2
    f2 = math.sqrt(ProcessNum) * (((Real_Distri_P - Inverted_Distri_P_tem)**2).sum(axis = 0)) # Object function 1
    return f1,f2,f1+f2

#----------------------------------------------
@jit
def gradient(DataSet_A, Real_Distri_P, Pars_mu, grad_StepSize = 0.001):
    grad = np.zeros(shape = (Pars_mu.shape),dtype = float) # i*s
    for i in range(Pars_mu.shape[0]):
        for s in range(Pars_mu.shape[1]):
            Pars_mu_tem_up = Pars_mu.copy()
            Pars_mu_tem_down = Pars_mu.copy()
            # X +/- Step
            Pars_mu_tem_up[i,s] += grad_StepSize
            Pars_mu_tem_down[i,s] -= grad_StepSize
            # Normalization
            Pars_mu_tem_up[i,:] / (1 + grad_StepSize)   
            Pars_mu_tem_down[i,:] / (1 - grad_StepSize)
            # f(X +/- Step)
            f1_up,f2_up,cost_up = cost(DataSet_A,Real_Distri_P,Pars_mu_tem_up)
            f1_down,f2_down,cost_down = cost(DataSet_A,Real_Distri_P,Pars_mu_tem_down)
            # Gradient
            grad[i,s] = (f2_up - f2_down) / (2*grad_StepSize)  # Only use f2
    return grad

# Iteration
#----------------------------------------------
@jit
def Iteration(DataSet,Real_Distri_P, Pars_mu_Initial, grad_StepSize = 0.01,Iter_StepSize = 0.001,Iter_Num = 2000):
    init_time = time.time()
    # Initialization
    costs = []   # 2.3s /10000events
    Pars_mu = Pars_mu_Initial.copy()
    for num in np.arange(Iter_Num):
        print("Iteration:",num)
        t1 = time.time()
        # Gradient
        grads = gradient(DataSet, Real_Distri_P, Pars_mu, grad_StepSize)  # Gradient
        Grad_Abs_Max = np.abs(grads).max()
        gamma = 0.001 / Grad_Abs_Max           # Learning Ratio
        t2 = time.time()
        print("Gradient time:",t2 - t1,'s')

        # Updating Parameter
        Pars_mu = Pars_mu - gamma*grads

        # Normalizing
        Sum_SIPM_i = np.sum(Pars_mu,axis = 1)
        for i in range(Pars_mu.shape[0]):
            Pars_mu[i,:] = Pars_mu[i,:] / Sum_SIPM_i[i]
        
        # Collecting
        costs.append(cost(DataSet,Real_Distri_P,Pars_mu))
        print(costs[num])
        
    return Pars_mu, costs, time.time() - init_time
        
        
# IterNUM = 1
# Pars_mu_end, cost_end, time_end = Iteration(DataSet_A, Real_Distri_P, Pars_mu_Initial, grad_StepSize = 0.1,Iter_StepSize = 0.001,Iter_Num = IterNUM)

# tt1 = time.time()
# print('costs:',cost(DataSet_A,Real_Distri_P,Pars_mu_Initial))
# print(time.time() - tt1,'s')

In [6]:
# CALCULATING PROCESS
#===========================================================================
# <<Crastal Cell Numbers and SIPM Numbers>>
#----------------------------------------------
PositionX_Num = 2
PositionY_Num = 2
PositionZ_Num = 2
PositionNumsXYZ =  np.array([PositionX_Num,PositionY_Num,PositionZ_Num])
Position_Nums = PositionX_Num * PositionY_Num * PositionZ_Num
Channel_Nums = 6*6*2

# Crastal and SIPM cells' centers coordination
#----------------------------------------------
Crastal_X,Crastal_Y,Crastal_Z = Cal_Crastal_Sites(PositionNumsXYZ)
SIPM_X,SIPM_Y,SIPM_Z = Cal_SIPM_Sites()
Coor_Cells = np.array([Crastal_X,Crastal_Y,Crastal_Z])
Coor_SIPMs = np.array([SIPM_X,SIPM_Y,SIPM_Z])
print("==================================================================")
print("---------------------------------1--------------------------------")
print("==================================================================")
print("Calculating Crastal and SIPM cells' centers coordination as:")
print("Coor_Cells[Crastal_X,Crastal_Y,Crastal_Z] and Coor_SIPMs[SIPM_X,SIPM_Y,SIPM_Z]")
print("Crastal coor's shape:",Coor_Cells.shape)
print("SIPM coor's shape:",Coor_SIPMs.shape)
print("Index sorted order: X -> Y -> Z")

---------------------------------1--------------------------------
Calculating Crastal and SIPM cells' centers coordination as:
Coor_Cells[Crastal_X,Crastal_Y,Crastal_Z] and Coor_SIPMs[SIPM_X,SIPM_Y,SIPM_Z]
Crastal coor's shape: (3, 8)
SIPM coor's shape: (3, 72)
Index sorted order: X -> Y -> Z


In [3]:
# Loading Data
#----------------------------------------------
ProcessNum = 1000
Num_npy = '_{:.0f}.npy'.format(ProcessNum)
#DataSet_A
DataSet_A = np.load('data/Result_Gamma/Gamma_data_DataSetA' + Num_npy)
DataSet_A_GridID = np.load('data/Result_Gamma/Gamma_data_DataSetA_GridID' + Num_npy)# 
#Real IPD
Real_Distri_P = np.load("data/Result_Gamma/Gamma_data_Real_IPD" + Num_npy) 

print("==================================================================")
print("---------------------------------2--------------------------------")
print("==================================================================")
print("Loading files: w_is(simulation), DataSet_A,DataSet_A_GridID,Real_IPD")
print("shapes:")
print("DataSet_A.shape:",DataSet_A.shape)
print("Real_IPD.shape:",Real_Distri_P.shape)
print("DataSet_A_GridID.shape:",DataSet_A_GridID.shape)


---------------------------------2--------------------------------
Loading files: w_is(simulation), DataSet_A,DataSet_A_GridID,Real_IPD
shapes:
DataSet_A.shape: (1000, 72)
Real_IPD.shape: (8,)
DataSet_A_GridID.shape: (1000,)


In [13]:
# Initializing Parameter mu_is
#----------------------------------------------
Pars_mu_Initial = np.zeros(shape = (Position_Nums,Channel_Nums),dtype = float) # mu_is ---->theta
t2_1 = time.time()
Pars_mu_Initial = Pars_Initialization(Coor_Cells,Coor_SIPMs)
t2_2 = time.time()
print("==================================================================")
print("---------------------------------3--------------------------------")
print("==================================================================")
print("Initializing w_is as Pars_mu_Initial:")
print('Time Exhausted:',t2_2 - t2_1,'s')
print(Pars_mu_Initial.shape)

---------------------------------3--------------------------------
Initializing w_is as Pars_mu_Initial:
Time Exhausted: 1.3928191661834717 s
(8, 72)


In [14]:
# Iteration
#----------------------------------------------
print("==================================================================")
print("---------------------------------5--------------------------------")
print("==================================================================")
print("Start Iterating.......................")
IterNUM = 200
Pars_mu_end, cost_end, time_end = Iteration(DataSet_A, Real_Distri_P, Pars_mu_Initial, grad_StepSize = 0.01,Iter_StepSize = 0.001,Iter_Num = IterNUM)

---------------------------------5--------------------------------
Start Iterating.......................
Iteration: 0
Gradient time: 15.346626996994019 s
(10.96027368635599, 0.36480032978162436, 11.325074016137615)
Iteration: 1
Gradient time: 12.281055927276611 s
(10.910448017279105, 0.3486094698504007, 11.259057487129505)
Iteration: 2
Gradient time: 12.45864486694336 s
(10.898596942562467, 0.3410200018035462, 11.239616944366013)
Iteration: 3
Gradient time: 12.062387704849243 s
(10.888221651329772, 0.31856783240676745, 11.20678948373654)
Iteration: 4
Gradient time: 11.928635597229004 s
(10.885732923693563, 0.2710704206165545, 11.156803344310118)
Iteration: 5
Gradient time: 12.42252254486084 s
(10.890747755195608, 0.19802180912452802, 11.088769564320136)
Iteration: 6
Gradient time: 13.775267124176025 s
(10.896404498138024, 0.15501483360594062, 11.051419331743965)
Iteration: 7
Gradient time: 11.441690683364868 s
(10.875601773132258, 0.10384918720907411, 10.979450960341332)
Iteration: 8


Gradient time: 9.71120285987854 s
(10.364767369500695, 0.0013281552526310332, 10.366095524753327)
Iteration: 72
Gradient time: 9.382336616516113 s
(10.387909904370797, 0.001075174845517627, 10.388985079216315)
Iteration: 73
Gradient time: 9.273975610733032 s
(10.402056515381656, 0.0034152576978504897, 10.405471773079507)
Iteration: 74
Gradient time: 8.662984371185303 s
(10.437628592411158, 0.001391402988507339, 10.439019995399665)
Iteration: 75
Gradient time: 9.103201866149902 s
(10.45333000132891, 0.004174204645408657, 10.457504205974319)
Iteration: 76
Gradient time: 8.61325216293335 s
(10.475098180704858, 0.0011384214504753121, 10.476236602155334)
Iteration: 77
Gradient time: 9.12068486213684 s
(10.489250503506923, 0.002529821095231149, 10.491780324602155)
Iteration: 78
Gradient time: 9.223242998123169 s
(10.504075359730557, 0.0013914035615061075, 10.505466763292063)
Iteration: 79
Gradient time: 9.28278112411499 s
(10.51817440208833, 0.003288768042789317, 10.52146317013112)
Iteration

Gradient time: 9.426213502883911 s
(12.200468276532051, 0.008032185991926911, 12.208500462523979)
Iteration: 145
Gradient time: 9.250487089157104 s
(12.242901316574718, 0.011447441929311967, 12.25434875850403)
Iteration: 146
Gradient time: 8.895272970199585 s
(12.28904729272098, 0.00847490418956904, 12.297522196910549)
Iteration: 147
Gradient time: 8.85227084159851 s
(12.335391933503548, 0.013218318029702308, 12.34861025153325)
Iteration: 148
Gradient time: 8.625842332839966 s
(12.388747202282637, 0.01005604232979287, 12.39880324461243)
Iteration: 149
Gradient time: 8.87087893486023 s
(12.449443771078613, 0.012016652443443757, 12.461460423522057)
Iteration: 150
Gradient time: 8.625892162322998 s
(12.503928678686933, 0.01075174300412948, 12.514680421691063)
Iteration: 151
Gradient time: 9.507263422012329 s
(12.57570138523896, 0.011384197785987146, 12.587085583024946)
Iteration: 152
Gradient time: 8.617122173309326 s
(12.644320024850078, 0.011384197785987146, 12.655704222636064)
Iteratio

In [15]:
# Saving Results
#----------------------------------------------
Num_npy1 = '_{:.0f}_{:.0f}.npy'.format(ProcessNum,IterNUM)
np.save('data/Results_Iteration/Pars_mu_end_f2' + Num_npy1,Pars_mu_end)
np.save('data/Results_Iteration/cost_end_f2' + Num_npy1,cost_end)



print("Saving Pars_mu_end,Pars_mu_Initial,Costs_end as npym files")
print(time_end,"s")      # 25s/Iteration

Saving Pars_mu_end,Pars_mu_Initial,Costs_end as npym files
2043.4457249641418 s


### Check codes:

In [36]:
# Initializing the Inverted IPD
#----------------------------------------------
t3_1 = time.time()
GridID_initial, Inverted_Distri_P_Initial = model(DataSet_A,Pars_mu_Initial)
t3_2 = time.time()
print("==================================================================")
print("---------------------------------4--------------------------------")
print("==================================================================")
print("Iverting IPD as Inverted_Distri_P_Initial")
print("Inverted_Distri_P_Initial.shape:",Inverted_Distri_P_Initial.shape,GridID_initial.shape)
print('Time Exhausted:',t3_2 - t3_1,'s')

Starting griding...........
Running Time: 0.004985809326171875 s
Process Gamma num is: 1000
IPD cell counts / Cells = 8 / 8
---------------------------------4--------------------------------
Iverting IPD as Inverted_Distri_P_Initial
Inverted_Distri_P_Initial.shape: (8,) (1000,)
Time Exhausted: 0.007977485656738281 s
